In [ ]:
import numpy as np
from skimage import io

from matplotlib import pyplot as plt

from matplotlib import cm
import networkx as nx
from tqdm.notebook import tqdm

from bresenham import bresenham
from matplotlib import cm, colors
import matplotlib.patches as mpatches
from collections import Counter
import pandas as pd

import matplotlib.pyplot as plt

import itertools

from mpire import WorkerPool

import shutil
from mpl_toolkits.axes_grid1 import make_axes_locatable
from pathlib import Path
import json

from wc_cv.src.utils import grainPreprocess, grainMark, SEMDataset, grainDraw, Crack

from pathos.multiprocessing import ProcessingPool as Pool  # Alternative parallelization
import copy

import graph_tool.all as gt
from itertools import islice

In [ ]:
eps = 300
# image = io.imread('wc_cv/cv/plots/WC+8Co_5_crack.jpg')[200:-300]
image = io.imread('wc_cv/cv/plots/WC+8Co_5_crack.jpg')[200:-300]
# image = io.imread('wc_cv/cv/plots/WC+8Co_5_fixed_001_cropped.jpg')

border=30

(entry_nodes,
 exit_nodes,
 img_contours,
 img_preprocessed_final,
 cnts,
 nodes_metadata)  = Crack.preprocess_graph_image(image, border=border, disk=5)

plot=False
# plot=True

if plot:
    fig, axes = plt.subplots(1,2,figsize=(20,20))
    axes[0].imshow(img_contours)
    axes[1].imshow(img_preprocessed_final)
    # plt.savefig('miniature_figures_aligned_crack.jpg', bbox_inches='tight')

    # axes[0].set_ylim(1000,1700)
    # axes[1].set_ylim(1000,1700)
    plt.show()

g, img_contours =  Crack.create_crack_graph(img_preprocessed_final.shape, cnts, nodes_metadata, eps=eps)

In [ ]:
save=False
# save=True

Crack.Viz.graph_plot(g, img_preprocessed_final, name='test.jpg', save=save)

In [ ]:
%%time

# entry_nodes = [ 0,3,4, 11]
# exit_nodes = [ 84, 30, 31]

# WC+8Co_5_fixed_001_cropped.jpg
entry_nodes = [ 4, 15, 13, 16, 34, 27]
exit_nodes = [ 527, 528, 519, 522]

# WC+8Co_5_crack
# entry_nodes = [ 4, 11, 16, 61, 62, 66]
# exit_nodes = [ 307, 308, 515, 529]

# edge types
# 0 - Co
# 1 - WC-Co
# 2 - WC
# 3 - WC-WC

co_co_e_max=20
wc_co_e_max=20
first_k_paths=1

parallel=True
# parallel=False
workers=20

param_1_max=15 # Co
param_2_max=10 # WC-Co
param_3_const=20 # WC
param_4_const=0 # WC-WC

energy_conf=np.zeros((param_1_max, param_2_max)).tolist()

for i,en_1 in enumerate(tqdm(range(0,param_1_max))):
    for j,en_2 in enumerate(range(0,param_2_max)):

        energy_conf[i][j]={
            0: en_1, # Co
            1: en_2, # WC-Co
            2: param_3_const, # WC
            3: param_4_const # WC-WC
            } 

energies_paths = Crack.Energy.get_energies(energy_conf,
                                            g,
                                            cnts,
                                            nodes_metadata,
                                            entry_nodes,
                                            exit_nodes,
                                            first_k_paths=first_k_paths,
                                            parallel=parallel,
                                            workers=workers)


In [ ]:
# name='wc_cv/cv/WC+8Co_5_fixed_001_cropped_optimized_e.jpg'
name='wc_cv/cv/WC+8Co_5_crack_fixed_optimized_e_labeled_no_wc_10.jpg'

# save=True
save=False

path_index=0

Crack.Viz.plot_optimized_energies(  energies_paths,
                                    path_index=path_index,
                                    N=5,M=5,
                                    y_label ='co_co_e',
                                    x_label = 'wc_co_e',
                                    name=name)

# Fixed paths energies 

In [ ]:
%%time
# df['index'] = range(len(df))
# df.set_index('index')

# WC+8Co_5_crack
# entry_nodes = [ 4, 11, 16, 61, 62, 66]
# exit_nodes = [ 307, 308, 515, 529]

# WC+8Co_5_fixed_001_cropped.jpg
entry_nodes = [ 4, 15, 13, 16, 34, 27]
exit_nodes = [ 527, 528, 519, 522]

workers=23
np.random.seed(51)
first_k_paths=1
parallel=True

#################################
# calculate base paths energies # 
#################################

param_1_const=15 # Co
param_2_const=15 # WC-Co
param_3_const=20 # WC
param_4_const=0 # WC-WC

energy_conf=np.zeros((1, 1)).tolist()

energy_conf[0][0]={
    0: param_1_const, # Co
    1: param_2_const, # WC-Co
    2: param_3_const, # WC
    3: param_4_const # WC-WC
    } 

energies_paths = Crack.Energy.get_energies( energy_conf,
# energies_paths = get_energies( energy_conf,
                                            g,
                                            cnts,
                                            nodes_metadata,
                                            entry_nodes,
                                            exit_nodes,
                                            first_k_paths=first_k_paths,
                                            parallel=parallel,
                                            workers=workers)

########################
# recalculate energies # 
########################

param_1_max= 20 # Co
param_2_max= 20 # WC-Co
param_3_const=20 # WC
param_4_const=0 # WC-WC

parallel=False

energy_conf=np.zeros((param_1_max, param_2_max)).tolist()

for i,en_1 in enumerate(tqdm(range(0,param_1_max))):
    for j,en_2 in enumerate(range(0,param_2_max)):

        energy_conf[i][j]={
            0: en_1, # Co
            1: en_2, # WC-Co
            2: param_3_const, # WC
            3: param_4_const # WC-WC
            } 

energies_paths_recalculated = Crack.Energy.get_energies( energy_conf,
# energies_paths_recalculated =    get_energies( energy_conf,
                                            g,
                                            cnts,
                                            nodes_metadata,
                                            entry_nodes,
                                            exit_nodes,
                                            first_k_paths=first_k_paths,
                                            parallel=parallel,
                                            workers=workers,
                                            recalculate_paths=energies_paths)

In [ ]:
# name='wc_cv/cv/WC+8Co_5_fixed_001_cropped_optimized_e.jpg'
name='wc_cv/cv/WC+8Co_5_crack_fixed_optimized_e_labeled_no_wc_10.jpg'

# save=True
save=False

path_index=0

Crack.Viz.plot_optimized_energies(  energies_paths_recalculated,
                                    path_index=path_index,
                                    N=5,M=5,
                                    y_label ='co_co_e',
                                    x_label = 'wc_co_e',
                                    name=name,
                                    save=save,
                                    fixed_paths=True
                                    )

# Dont know

In [ ]:
fig, axes = plt.subplots(1,1,figsize=(15,20))
border=30

pos = nx.get_node_attributes(g, 'pos')

df_shortest_entry = copy.copy(df)

x=329
y=33

# df_shortest_entry=df_shortest_entry[df_shortest_entry['path'].apply(lambda arr: x in arr)]
# df_shortest_entry=df_shortest_entry[df_shortest_entry['path'].apply(lambda arr: y in arr)]

cmap = cm.get_cmap("Spectral")
# need to normalize because color maps are defined in [0, 1]
norm = colors.Normalize(df_shortest_entry['path_len_pixel'].values.min(), df_shortest_entry['path_len_pixel'].values.max())


for i in range(len(df_shortest_entry)):
    line = df_shortest_entry.iloc[i]
    path = line['path']
    
    path_edges = list(zip(path,path[1:]))

    nx.draw_networkx_edges(g,pos,edgelist=path_edges,edge_color=cmap(norm(line['path_len_pixel'])),width=2)
    nx.draw_networkx_nodes(g,pos,nodelist=[line['entry_node']],node_color='blue',node_size = 100)
    nx.draw_networkx_nodes(g,pos,nodelist=[line['exit_node']],node_color='red',node_size = 100)


    labels = {line['entry_node']: line['entry_node']}
    nx.draw_networkx_labels(g, pos,labels, font_size=20, font_color='orange')

    labels = {line['exit_node']: line['exit_node']}
    nx.draw_networkx_labels(g, pos,labels, font_size=20,font_color='black')



im = axes.imshow(img_aligned, cmap='gray')

plt.axis("on")
axes.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

axes.arrow(12, border-5, 0, 200, width=0.3, length_includes_head=True, head_width=10, head_length=10,color=(0,0,0))

axes.text(2, img_aligned.shape[0]*2/3,'hit direction',rotation=90, color=(0, 0, 0),fontsize=15)
axes.text(img_aligned.shape[1]/2-50, border/3,'entry nodes', color=(0, 0, 1),fontsize=15)
axes.text(img_aligned.shape[1]/2-25,img_aligned.shape[0] - border/3,'exit nodes', color=(1, 0, 0),fontsize=15)


fig.colorbar(cm.ScalarMappable(norm=norm, cmap=cmap), ax=axes)

# plt.savefig('shortests_paths_crack.jpg',bbox_inches='tight')

plt.show()

# Shortest paths

In [ ]:
# 'path':all_paths,
# 'path_len_edges':all_path_len_edges,
# 'path_len_pixel':all_path_len_pixels,
# 'entry_node':all_entry_nodes,
# 'exit_node':all_exit_nodes

entry_nodes = np.unique(df['entry_node'])
exit_nodes = np.unique(df['exit_node'])

N=1

shortest_entry_paths = []
for entry_node in tqdm(entry_nodes):
    # for i in range(N):
    row = df[df.entry_node==entry_node].sort_values(by='path_len_pixel').iloc[0]
    shortest_entry_paths.append(row)

shortest_exit_paths = []
for exit_node in tqdm(exit_nodes):
    # for i in range(N):
    row = df[df.exit_node==exit_node].sort_values(by='path_len_pixel').iloc[0]
    shortest_exit_paths.append(row)

df_shortest_entry = pd.DataFrame(shortest_entry_paths)
df_shortest_exit = pd.DataFrame(shortest_exit_paths)

In [ ]:
# df['path'] = df['path'].apply(lambda x: np.array(x))
# df.to_csv('shortest_paths_edges.csv')

df1=pd.read_csv('shortest_paths_edges.csv')

entry_nodes = np.unique(df['entry_node'])
exit_nodes = np.unique(df['exit_node'])

N=10

shortest_entry_paths = []
for entry_node in tqdm(entry_nodes):
    for i in range(N):
        row = df[df.entry_node==entry_node].sort_values(by='path_len_pixel').iloc[i]
        shortest_entry_paths.append(row)


df1_shortest_entry = pd.DataFrame(shortest_entry_paths)


In [ ]:
df=pd.read_csv('paths.csv')

entry_nodes = np.unique(df['entry_node'])
exit_nodes = np.unique(df['exit_node'])

shortest_entry_paths = []
for entry_node in tqdm(entry_nodes):
    row = df[df.entry_node==entry_node].sort_values(by='path_len_pixel').iloc[0]
    shortest_entry_paths.append(row)

shortest_exit_paths = []
for exit_node in tqdm(exit_nodes):
    row = df[df.exit_node==exit_node].sort_values(by='path_len_pixel').iloc[0]
    shortest_exit_paths.append(row)

df_shortest_entry = pd.DataFrame(shortest_entry_paths)
df_shortest_exit = pd.DataFrame(shortest_exit_paths)



fig,axes = plt.subplots(1,1,figsize=(15,15))

pos = nx.get_node_attributes(g, 'pos')

for i in range(len(df_shortest_entry)):
    line = df_shortest_entry.iloc[i]
    path = line['path']
    path_edges = list(zip(path,path[1:]))

    nx.draw_networkx_edges(g,pos,edgelist=path_edges,edge_color='blue',width=1)
    nx.draw_networkx_nodes(g,pos,nodelist=[line['entry_node']],node_color='blue',node_size = 100)

for i in range(len(df_shortest_exit)):
    line = df_shortest_exit.iloc[i]
    path = list(reversed(line['path']))
    path_edges = list(zip(path,path[1:]))

    nx.draw_networkx_edges(g,pos,edgelist=path_edges,edge_color='r',width=1)
    nx.draw_networkx_nodes(g,pos,nodelist=[line['exit_node']],node_color='r',node_size = 100)

axes.imshow(img_aligned, cmap='gray')

plt.axis("on")
axes.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

axes.arrow(12, border-5, 0, 200, width=0.3, length_includes_head=True, head_width=10, head_length=10,color=(0,0,0))

axes.text(2, img_aligned.shape[0]*2/3,'hit direction',rotation=90, color=(0, 0, 0),fontsize=15)
axes.text(img_aligned.shape[1]/2-25, border/3,'entry nodes', color=(0, 0, 1),fontsize=15)
axes.text(img_aligned.shape[1]/2-25,img_aligned.shape[0] - border/3,'exit nodes', color=(1, 0, 0),fontsize=15)

# plt.savefig('shortests_paths.jpg',bbox_inches='tight')

plt.show()

# Deprecated

In [ ]:
import networkx as nx
from graph_tool.all import Graph

def nx_to_gt_with_attrs(nx_graph):
    gt_graph = Graph(directed=nx_graph.is_directed())
    vprop_name = gt_graph.new_vertex_property("string")
    eprop_weight = gt_graph.new_edge_property("float")

    nx_to_gt_v = {}
    for node in nx_graph.nodes(data=True):
        v = gt_graph.add_vertex()
        nx_to_gt_v[node[0]] = v
        if 'name' in node[1]:
            vprop_name[v] = str(node[1]['name'])

    for u, v, data in nx_graph.edges(data=True):
        e = gt_graph.add_edge(nx_to_gt_v[u], nx_to_gt_v[v])
        if 'weight' in data:
            eprop_weight[e] = float(data['weight'])

    gt_graph.vertex_properties['name'] = vprop_name
    gt_graph.edge_properties['weight'] = eprop_weight

    return gt_graph

gt_graph = nx_to_gt_with_attrs(g_weighed)

gt.graph_draw(gt_graph, vertex_text=gt_graph.vertex_index, output="wc_cv/cv/two-nodes.png")

# eweight = gt_graph.new_edge_property("weight")
eweight = gt_graph.new_edge_property("float")

vlist = gt_graph.get_vertices()
edges = gt_graph.get_edges()

paths =  list(gt.all_shortest_paths(gt_graph, source=entry_node, target=exit_node, weights=eweight))

# eweight = gt_graph.new_edge_property("weight")
eweight = gt_graph.new_edge_property("float")

vlist = gt_graph.get_vertices()
edges = gt_graph.get_edges()

paths =  list(gt.all_shortest_paths(gt_graph, source=entry_node, target=exit_node, weights=eweight))